In [0]:
import os
from os import listdir
from os.path import isfile, join

In [2]:
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
from keras.models import Model
from keras.layers import Dense, Dropout, GlobalAveragePooling2D, BatchNormalization, Activation
from keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from keras.optimizers import Adam
from keras.applications import resnet50
from keras.utils import to_categorical
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!ls "/content/drive/My Drive/Colab Notebooks/experiments"

cifar10_cnn.ipynb	dataset
cifar10_resnet50.ipynb	transfer_learing_keras.ipynb


In [5]:
parent_dir = '/content/drive/My Drive/Colab Notebooks/experiments/dataset'
sub_dirs = next(os.walk(parent_dir))[1]
print (sub_dirs)

['person', 'bicycle', 'cow', 'bus', 'car', 'bird', 'diningtable', 'cat', 'horse', 'chair', 'dog', 'bottle', 'aeroplane', 'pottedplant', 'boat', 'motorbike']


We have 16 folders containing images from the above mentioned categories.  
Now we will create a dataframe containing category, file_name, and encoding of the catgeory.

In [0]:
files_details = []
for sub_dir in sub_dirs:
    dir_path = join(parent_dir, sub_dir)
    files = [f for f in listdir(dir_path) if isfile(join(dir_path, f))]
    for file in files:
        file_detail = {}
        file_detail['class'] = sub_dir
        file_detail['path'] = join(parent_dir, sub_dir, file)
        if os.path.getsize(file_detail['path'])>0:
            files_details.append(file_detail)
        else:
            pass
        
lbl = LabelEncoder()
df = pd.DataFrame(files_details)
df['class_encoded'] = lbl.fit_transform(df['class'])

In [7]:
df.head()

,class,path,class_encoded
0,person,/content/drive/My Drive/Colab Notebooks/experi...,14
1,person,/content/drive/My Drive/Colab Notebooks/experi...,14
2,person,/content/drive/My Drive/Colab Notebooks/experi...,14
3,person,/content/drive/My Drive/Colab Notebooks/experi...,14
4,person,/content/drive/My Drive/Colab Notebooks/experi...,14


In [8]:
df.shape, df['class'].nunique()

((793, 3), 16)

It is a small dataset. 793 images are present and the number of categories present are 16

In [9]:
df['class'].value_counts()

person         50
diningtable    50
aeroplane      50
bird           50
dog            50
car            50
boat           50
bus            50
cow            50
bicycle        50
cat            50
horse          50
bottle         50
motorbike      50
chair          50
pottedplant    43
Name: class, dtype: int64

The number of images belonging to a category is around 50

In [0]:
# function for getting the image values in the dataframe
def get_np_images(x):
    original_image = load_img(x, target_size=(256, 256))
    numpy_image = img_to_array(original_image)
    #input_image = np.expand_dims(numpy_image, axis=0)
    #processed_image_resnet50 = resnet50.preprocess_input(input_image.copy())
    return numpy_image

In [0]:
df['image'] = df['path'].apply(lambda x : get_np_images(x))

df.head()

In [12]:
resnet_model = resnet50.ResNet50(weights='imagenet', include_top=False, input_shape= (256,256,3))

W0812 06:29:19.191029 140420930078592 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0812 06:29:19.214984 140420930078592 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0812 06:29:19.221347 140420930078592 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0812 06:29:19.245777 140420930078592 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0812 06:29:19.246824

In [0]:
# freeze all the parameters of the resnet model
for layer in resnet_model.layers:
    layer.trainable=False

In [14]:
# add our own layers
x = resnet_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256)(x) 
x = BatchNormalization()(x)
x = Activation('tanh')(x)
x = Dropout(0.4)(x)
x = Dense(64)(x) 
x = BatchNormalization()(x)
x = Activation('tanh')(x)
x = Dropout(0.4)(x)
preds = Dense(16, activation='softmax')(x)

W0812 06:29:28.021996 140420930078592 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [15]:
model=Model(inputs=resnet_model.input,outputs=preds)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 128, 128, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 128, 128, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [16]:
[(layer, layer.trainable) for layer in model.layers]

[(<keras.engine.input_layer.InputLayer at 0x7fb5f8531048>, False),
 (<keras.layers.convolutional.ZeroPadding2D at 0x7fb5f8531400>, False),
 (<keras.layers.convolutional.Conv2D at 0x7fb5f85311d0>, False),
 (<keras.layers.normalization.BatchNormalization at 0x7fb5f85316d8>, False),
 (<keras.layers.core.Activation at 0x7fb5f8531fd0>, False),
 (<keras.layers.convolutional.ZeroPadding2D at 0x7fb5ea682a20>, False),
 (<keras.layers.pooling.MaxPooling2D at 0x7fb5f7d25dd8>, False),
 (<keras.layers.convolutional.Conv2D at 0x7fb5f855fdd8>, False),
 (<keras.layers.normalization.BatchNormalization at 0x7fb5ea667ac8>, False),
 (<keras.layers.core.Activation at 0x7fb5ea667b70>, False),
 (<keras.layers.convolutional.Conv2D at 0x7fb5ea5df4a8>, False),
 (<keras.layers.normalization.BatchNormalization at 0x7fb5ea624f98>, False),
 (<keras.layers.core.Activation at 0x7fb5ea5805c0>, False),
 (<keras.layers.convolutional.Conv2D at 0x7fb5ea50fbe0>, False),
 (<keras.layers.convolutional.Conv2D at 0x7fb5ea4e590

In [17]:
# divide the dataframe into training and validation dataframe
df = df.sample(frac=1, random_state=12).reset_index(drop=True)
train, valid = train_test_split(df, test_size=0.2, random_state = 12)
train.reset_index(drop = True, inplace = True)
valid.reset_index(drop = True, inplace = True)
train.shape, valid.shape

((634, 4), (159, 4))

In [0]:
# divide into x and y train and test
X_train, y_train = train['image'], train['class_encoded']
X_test, y_test = valid['image'], valid['class_encoded']

# make the Pandas series into numpy ndarray
X_train = np.array([image for image in X_train])
X_test = np.array([image for image in X_test])

# preprocess the images for feeding into the resnet model
X_train = resnet50.preprocess_input(X_train)
X_test = resnet50.preprocess_input(X_test)

# one hot encode the target variables
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [19]:
opt = Adam(lr=0.0001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

W0812 06:29:29.139939 140420930078592 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [20]:
model.fit(X_train, y_train, validation_split=0.2, epochs=10, batch_size = 32)

W0812 06:29:29.247617 140420930078592 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 507 samples, validate on 127 samples
Epoch 1/10
507/507 [==============================] - 10s 21ms/step - loss: 2.8792 - acc: 0.1302 - val_loss: 2.3482 - val_acc: 0.2520
Epoch 2/10
507/507 [==============================] - 3s 7ms/step - loss: 2.4761 - acc: 0.2091 - val_loss: 2.0641 - val_acc: 0.3307
Epoch 3/10
507/507 [==============================] - 3s 7ms/step - loss: 2.1129 - acc: 0.3195 - val_loss: 1.8834 - val_acc: 0.4094
Epoch 4/10
507/507 [==============================] - 3s 7ms/step - loss: 1.9143 - acc: 0.4379 - val_loss: 1.7663 - val_acc: 0.4882
Epoch 5/10
507/507 [==============================] - 3s 7ms/step - loss: 1.7323 - acc: 0.4970 - val_loss: 1.6829 - val_acc: 0.5039
Epoch 6/10
507/507 [==============================] - 3s 7ms/step - loss: 1.5803 - acc: 0.5444 - val_loss: 1.6100 - val_acc: 0.5039
Epoch 7/10
507/507 [==============================] - 3s 7ms/step - loss: 1.5307 - acc: 0.5621 - val_loss: 1.5611 - val_acc: 0.5039
Epoch 8/10
507/507 [========

In [21]:
model.evaluate(X_test, y_test)

159/159 [==============================] - 1s 6ms/step


[1.3499425304760724, 0.6352201227871876]

In [0]:
# divide into x and y train and test
X_train, y_train = train['image'], train['class_encoded']
X_test, y_test = valid['image'], valid['class_encoded']

# make the Pandas series into numpy ndarray
X_train = np.array([image for image in X_train])
X_test = np.array([image for image in X_test])

X_train_n, X_valid, y_train_n, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state = 12)

# preprocess the images for feeding into the resnet model
X_train_n = resnet50.preprocess_input(X_train_n)
X_valid = resnet50.preprocess_input(X_valid)
X_test = resnet50.preprocess_input(X_test)

# one hot encode the target variables
y_train_n = to_categorical(y_train_n)
y_valid = to_categorical(y_valid)
y_test = to_categorical(y_test)

In [23]:
X_train_n.shape, X_valid.shape, y_train_n.shape, y_valid.shape

((507, 256, 256, 3), (127, 256, 256, 3), (507, 16), (127, 16))

In [0]:
datagen = ImageDataGenerator(
        width_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

In [0]:
datagen.fit(X_train_n)

In [26]:
model.fit_generator(datagen.flow(X_train_n, y_train_n,
                    batch_size=32),
                    samples_per_epoch=X_train_n.shape[0],
                    nb_epoch=30,
                    validation_data=(X_valid, y_valid))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., validation_data=(array([[[..., steps_per_epoch=15, epochs=30)`
  """


Epoch 1/30
15/15 [==============================] - 7s 456ms/step - loss: 1.4768 - acc: 0.5750 - val_loss: 0.9452 - val_acc: 0.7638
Epoch 2/30
15/15 [==============================] - 6s 417ms/step - loss: 1.4353 - acc: 0.6121 - val_loss: 0.9330 - val_acc: 0.7638
Epoch 3/30
15/15 [==============================] - 6s 415ms/step - loss: 1.3644 - acc: 0.5891 - val_loss: 0.9323 - val_acc: 0.7402
Epoch 4/30
15/15 [==============================] - 6s 418ms/step - loss: 1.2794 - acc: 0.6500 - val_loss: 0.9187 - val_acc: 0.7480
Epoch 5/30
15/15 [==============================] - 6s 420ms/step - loss: 1.2036 - acc: 0.6530 - val_loss: 0.9141 - val_acc: 0.7559
Epoch 6/30
15/15 [==============================] - 6s 416ms/step - loss: 1.2651 - acc: 0.6628 - val_loss: 0.9078 - val_acc: 0.7480
Epoch 7/30
15/15 [==============================] - 6s 417ms/step - loss: 1.1827 - acc: 0.6534 - val_loss: 0.9024 - val_acc: 0.7480
Epoch 8/30
15/15 [==============================] - 6s 415ms/step - loss: 1.

In [27]:
model.evaluate(X_test, y_test)

159/159 [==============================] - 1s 6ms/step


[1.0840155276112586, 0.7232704428756762]